In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import (
    FewShotChatMessagePromptTemplate,
    FewShotPromptTemplate,
)

"""
풀스택 GPT: #4.0부터 ~ #4.6까지 Model I/O
영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.
"""

chat = ChatOpenAI(
    # model_name="gpt-5-nano",
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "question": "영화 올드보이 에 대한 정보 찾아줘",
        "answer": (
            """
            "제목" : "올드보이",
            "감독" : "박찬욱", 
            "주요 출연진" : "최민식,유지태,강혜정,김병옥",
            "영화의 장르" : "코메디, 드라마, 범죄, 스릴러, 미스테리", 
            "시놉시스" : " 술에 취해 집에 가는 길, 누군가에게 납치당해 사설 감금방에 갇힌 오대수(최민식)는 감금 1년 무렵, TV를 통해 아내가 살해당하고 자신이 용의자로 지목되었음을 알게 된다. 복수와 탈출의 기회를 노리며 그곳에서 지내기를 15년. 
            어느 날, 잠든 대수에게 최면술사(이승신)가 찾아와 최면을 걸고, 대수는 자신이 납치당했던 곳에서 눈을 뜬다. 
            그는 우연히 한 일식집에서 요리사 미도(강혜정)를 만나고, 그녀를 사랑하게 된다. 마침내 대수는 자신을 납치한 우진(유지태)을 만나게 되지만, 
            우진은 5일 안에 감금의 원인을 스스로 알아내라며 미도와 자신의 목숨을 건 게임을 제안한다. 대수는 우진과 고등학교 동창 사이이며, 
            고등학생 시절 우진과 그의 누나 수아(윤진서)가 관계 맺는 것을 보고 친구에게 말한 것이 임신 소문으로 확산되어 수아가 자살했음을 알게 된다. 
            그러나 대수는 우진에게서 미도가 자신의 딸 연희임을 듣게 되고, 미도가 진실을 알게 될 것을 두려워한 대수는 우진 앞에서 가위로 자신의 혀를 자른다. 
            모든 복수를 마친 우진은 수아가 죽던 날, 다리에 매달린 누나의 손을 놓아버린 일을 회상하며 목숨을 끊는다. 눈 덮인 산 중턱, 대수는 최면술사에게 모든 것을 잊게 해달라고 하고, 
            얼마 후 최면에서 깨어난다. 눈밭에 덩그러니 누워 있는 그를 찾은 미도는 그의 언 몸을 녹여주며 그를 끌어안는다.",
            "개봉일" : "2003-11-21",
        """
        ),
    },
    {
        "question": "영화 부산행 에 대한 정보 찾아줘",
        "answer": (
            """
            "제목" : "부산행",
            "감독" : "연상호", 
            "주요 출연진" : "공유, 정유미, 마동석, 김수안, 김의성 ",
            "영화의 장르" : "액션, 스릴러, 재난", 
            "시놉시스" : " 전대미문의 재난이 대한민국을 덮친다!
            정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데,
            KTX에 몸을 실은 사람들은 단 하나의 안전한 도시 부산까지
            살아가기 위한 치열한 사투를 벌이게 되는데..
            서울에서 부산까지의 거리 442km
            지키고 싶은, 지켜야만 하는 사람들의 극한의 사투!
            살고 싶다면, 부산행 KTX에 탑승하라! ",
            "개봉일" : "2016-07-20",
        """
        ),
    },
    {
        "question": "영화 보스 에 대한 정보 찾아줘",
        "answer": (
            """
            "제목" : "보스",
            "감독" : "라희찬", 
            "주요 출연진" : "조우진, 정경호, 박지환, 이규형, 이성민, 오달수, 황우슬혜, 정유진, 고창석, 정규수, 주진모",
            "영화의 장르" : "액션, 코메디", 
            "시놉시스" : " 갑작스러운 보스의 죽음! 차기 보스는 누구?!
            보스 후보1. 차기 보스 0순위지만, 조직이 아닌 중식당 미미루로 전국구 평정을 꿈꾸는 ‘순태’(조우진)
            보스 후보2. 조직 내 입지는 충분하나 운명처럼 만난 탱고에 인생을 건, 차기 보스 유력자 ‘강표’(정경호)
            보스 후보3. 유일하게 보스를 갈망하지만, 그 누구도 보스감이라 생각하지 않는, 보스 부적격자 ‘판호’(박지환)
            그리고 미미루 배달원으로 잠입한 언더커버 경찰 ‘태규’(이규형)까지 가세하며 치열한 보스 ‘양보’ 전은 예측불허 대혼란으로 치닫는데..
            세력 전쟁보다 살벌한 보스 대결 양보 전쟁!
            올 추석, 웃기는 놈이 보스다! ",
            "개봉일" : "2025-10-03",
        """
        ),
    },
]


example_prompt = """
    Human:{question},
    AI:{answer}
"""
example_prompt = PromptTemplate.from_template(example_prompt)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: 영화 {movie}에 대한 정보 찾아줘",
    input_variables=["movie"],
)


# final_prompt.format(movie="괴물")


chain = prompt | chat

chain.invoke({"movie": "아가씨"})

AI:
        "제목" : "아가씨",
        "감독" : "박찬욱", 
        "주요 출연진" : "김민희, 김태리, 하정우, 조진웅",
        "영화의 장르" : "드라마, 로맨스, 스릴러",
        "시놉시스" : "일제 강점기 시대, 조선의 미국인 장가에 시집온 후 사망한 남편의 미망인 소희(김민희)는 남편의 친구이자 조선 최고의 도둑 소희(하정우)에게 불륜을 요구한다. 
        소희는 소희에게 불륜을 요구하는 대신 소희를 조선 최고의 가정부로 영입시키고, 소희의 가정부로서의 역량을 테스트하기 위해 그녀를 일본인 레이디 아가씨(김태리)의 가정부로 보낸다. 
        소희는 레이디 아가씨에게 점차 끌리게 되고, 레이디 아가씨 역시 소희에게 감정을 품게 되지만, 레이디 아가씨의 숨겨진 정체와 소희의 진짜 목적이 드러나면서 두 사람의 운명은 뒤바뀌게 된다.",
        "개봉일" : "2016-06-01",

AIMessageChunk(content='AI:\n        "제목" : "아가씨",\n        "감독" : "박찬욱", \n        "주요 출연진" : "김민희, 김태리, 하정우, 조진웅",\n        "영화의 장르" : "드라마, 로맨스, 스릴러",\n        "시놉시스" : "일제 강점기 시대, 조선의 미국인 장가에 시집온 후 사망한 남편의 미망인 소희(김민희)는 남편의 친구이자 조선 최고의 도둑 소희(하정우)에게 불륜을 요구한다. \n        소희는 소희에게 불륜을 요구하는 대신 소희를 조선 최고의 가정부로 영입시키고, 소희의 가정부로서의 역량을 테스트하기 위해 그녀를 일본인 레이디 아가씨(김태리)의 가정부로 보낸다. \n        소희는 레이디 아가씨에게 점차 끌리게 되고, 레이디 아가씨 역시 소희에게 감정을 품게 되지만, 레이디 아가씨의 숨겨진 정체와 소희의 진짜 목적이 드러나면서 두 사람의 운명은 뒤바뀌게 된다.",\n        "개봉일" : "2016-06-01",')